In [1]:
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

print("OpenCV version", cv2.__version__)
print("NumPy version", np.__version__)

OpenCV version 4.5.5
NumPy version 1.21.5


In [34]:
def onScaleFactor(val):
    global scale, scaleFactor
    scale=val
    scaleFactor=1+scale/100
    
def onSize(val):
    global size
    size=val
    
def onNeighbor(val):
    global N
    N=val
    
video=cv2.VideoCapture('./Video/Barber Shop.mp4')
winName='People Blur'
size=30
N=3
scale=100
scaleFactor=1+scale/100
fontFace_classifier=cv2.CascadeClassifier('./haarcascade/haarcascade_frontalface_alt2.xml')
sideFace_classifier=cv2.CascadeClassifier('./haarcascade/haarcascade_profileface.xml')

cv2.namedWindow(winName, flags=1|cv2.WINDOW_GUI_NORMAL)
cv2.createTrackbar('size', winName, size, 200, onSize)
cv2.setTrackbarMin('size', winName, 10)
cv2.createTrackbar('Neighbors', winName, N, 20, onNeighbor)
cv2.setTrackbarMin('Neighbors', winName, 1)
cv2.createTrackbar('ScaleFactor', winName, scale, 200, onScaleFactor)
cv2.setTrackbarMin('ScaleFactor', winName, 100)

while True:
    ret, frame=video.read()
    if ret==True:
        frame=cv2.resize(frame, None, fx=1280/frame.shape[1], fy=1280/frame.shape[1])
        F_face_blur=fontFace_classifier.detectMultiScale(frame, minSize=(size, size), minNeighbors=N, scaleFactor=scaleFactor)
        S_face_blur=sideFace_classifier.detectMultiScale(frame, minSize=(size, size), minNeighbors=N, scaleFactor=scaleFactor)
        for x, y, w, h in F_face_blur:
            cap=cv2.GaussianBlur(frame[y:(y+h), x:(x+w)], (45, 45), 4.)
            frame[y:(y+h), x:(x+w)]=cap
        for x, y, w, h in S_face_blur:
            cap=cv2.GaussianBlur(frame[y:(y+h), x:(x+w)], (45, 45), 4.)
            frame[y:(y+h), x:(x+w)]=cap
        cv2.imshow(winName, frame)
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    else:
        video.release()
        break
cv2.destroyAllWindows()